In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_practice').getOrCreate()

22/09/06 12:48:32 WARN Utils: Your hostname, MacBook-Pro-van-Ujjwal.local resolves to a loopback address: 127.0.0.1; using 192.168.1.4 instead (on interface en0)
22/09/06 12:48:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/06 12:48:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
from pyspark.ml.regression import LinearRegression

In [9]:
data = spark.read.csv("Ecommerce_Customers.csv", inferSchema=True, header= True)

In [10]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [11]:
data.describe()

DataFrame[summary: string, Email: string, Address: string, Avatar: string, Avg Session Length: string, Time on App: string, Time on Website: string, Length of Membership: string, Yearly Amount Spent: string]

In [12]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [13]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [14]:
data.head()[1]

'835 Frank TunnelWrightmouth, MI 82180-9605'

In [15]:
data.head()[0]

'mstephenson@fernandez.com'

In [16]:
data.show()[0]

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

TypeError: 'NoneType' object is not subscriptable

In [17]:
data.show()[10]

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

TypeError: 'NoneType' object is not subscriptable

In [18]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [21]:
data.columns



['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [22]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'], outputCol='features')

In [23]:
output = assembler.transform(data)

In [24]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [25]:
final_data = output.select("features",'Yearly Amount Spent')

In [26]:
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [27]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [28]:
train_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [30]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                355|
|   mean| 500.03306973349913|
| stddev|  80.27901752327742|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [31]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                145|
|   mean| 497.55365085450467|
| stddev|  77.14805629153196|
|    min| 298.76200786180766|
|    max|  744.2218671047146|
+-------+-------------------+



In [37]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [39]:
lr_model = lr.fit(train_data)

22/09/06 13:15:31 WARN Instrumentation: [1340be8c] regParam is zero, which might cause numerical instability and overfitting.


In [40]:
lr_model.coefficients

DenseVector([25.7093, 38.48, 0.4241, 62.0966])

In [41]:
lr_model.intercept

-1049.3319428681236

In [42]:
test_Results = lr_model.evaluate(test_data)

In [43]:
test_Results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  -21.94384086038542|
|  -7.792603352359492|
|-0.00599230582190...|
|  18.654002709663757|
|  -4.062780190028207|
|   2.979479538255589|
|  -8.760039002835526|
|  -17.89400091694006|
|  18.090035181038502|
|  17.620158152456554|
|  -3.381870044426762|
|    7.02363015517534|
|  -1.022233195513877|
|  -7.972564950920628|
|  -6.039419263988691|
| -12.259882343566915|
|  -9.042218901466583|
|  1.1649558185681599|
|   8.386741856479887|
|  -6.277028423191894|
+--------------------+
only showing top 20 rows

